In [26]:
import pandas as pd
import json
import numpy as np
import itertools
import time
import math
ttt = time.time()

In [27]:
# Creating a normalised demand profile from Elexon Class 1 data, using 2021 as a base year
# Downloaded from https://ukerc.rl.ac.uk/DC/cgi-bin/edc_search.pl/?WantComp=42

infile = "ProfileClass1.csv"

baseload_profile_df = pd.read_csv(infile)

demand_cols = [x for x in baseload_profile_df.columns.values if x!='Time' ]
baseload_profile_df['annual_avg_demand_kW'] = baseload_profile_df[demand_cols].mean(axis=1)


baseload_profile_df['electricity_demand_normalised'] = baseload_profile_df['annual_avg_demand_kW'] / baseload_profile_df['annual_avg_demand_kW'].sum()
baseload_profile_df[['hour', 'minute']] = baseload_profile_df['Time'].str.split(':', 1, expand=True).astype(int)

baseload_profile_df['profile_id'] = 0
baseload_profile_df['profile_name'] = 'Elexon Class 1'

keep_cols = ['profile_id','profile_name','hour','minute','electricity_demand_normalised']
drop_cols = [x for x in baseload_profile_df.columns.values if x not in keep_cols]

baseload_profile_df.drop(labels=drop_cols, inplace=True, axis=1)

hh = pd.date_range("2021-01-01T00:00:00", "2021-12-31T23:30:00", freq="30min")
half_hourly_df = pd.DataFrame(data={'datetime':pd.date_range("2021-01-01T00:00:00", 
                                                             "2021-12-31T23:30:00", 
                                                             freq="30min")}
                             )

ds = half_hourly_df['datetime'].values


half_hourly_df['day_of_year'] = half_hourly_df['datetime'].dt.dayofyear
half_hourly_df['month'] = half_hourly_df['datetime'].dt.month
half_hourly_df['hour'] = half_hourly_df['datetime'].dt.hour
half_hourly_df['minute'] = half_hourly_df['datetime'].dt.minute

half_hourly_df = pd.merge(half_hourly_df, baseload_profile_df,
                          on=['hour','minute'], how='left')

print ('half_hourly_df Mem Usage:',half_hourly_df.memory_usage().sum()/1e6)
profile_grouped_df = half_hourly_df.groupby('profile_id')['electricity_demand_normalised'].sum().reset_index()
profile_grouped_df



half_hourly_df Mem Usage: 1.26144


,profile_id,electricity_demand_normalised
0,0,365.0


In [28]:
# # From Elexon Guidance:
# # https://www.elexon.co.uk/documents/training-guidance/bsc-guidance-notes/load-profiles/

# # DEFINITIONS OF SEASONS					
# # The year is broken down into the following seasons:					
					
# # (a) 	Spring:  defined as the period from the day of clock change from GMT to BST in March,				
# # 	up to and including the Friday preceding the start of the summer period;				
					
# # (b) 	Summer:  defined as the ten-week period, preceding High Summer, starting on the 				
# # 	sixteenth Saturday before the August bank Holiday; 				
					
# # (c) 	High Summer:  defined as the period of six weeks and two days from the sixth Saturday				
# # 	before August Bank Holiday up to and including the Sunday following August Bank Holiday; 				
					
# # (d) 	Autumn:  defined as the period from the Monday following the August Bank Holiday, 				
# # 	up to and including the day preceding the clock change from BST to GMT in October; 				
					
# # (e) 	Winter:  defined as the period from the day of clock change from British Summer Time				
# # 	(BST) to Greenwich Mean Time (GMT) in October, up to and including the day preceding			
# # 	the clock change from GMT to BST in March; 				

In [29]:

# Generating combinations 

annual_electricity_consumption_kWh = np.array([2200,2900,3600])
annual_gas_consumption_kWh = np.array([10000,12000,14000])

daily_miles_driven = np.array([0,20,50,100])

a = [annual_electricity_consumption_kWh,
     annual_gas_consumption_kWh,
     daily_miles_driven
    ]

energy_consumption_levels = list(itertools.product(*a))
energy_consumption_levels_df = pd.DataFrame(data=energy_consumption_levels,columns=['annual_electricity_consumption_kWh',
                                              'annual_gas_consumption_kWh','daily_miles_driven'
                                             ])
energy_consumption_levels_df

,annual_electricity_consumption_kWh,annual_gas_consumption_kWh,daily_miles_driven
0,2200,10000,0
1,2200,10000,20
2,2200,10000,50
3,2200,10000,100
4,2200,12000,0
5,2200,12000,20
6,2200,12000,50
7,2200,12000,100
8,2200,14000,0
9,2200,14000,20


In [30]:
# Loading in Vehicles data JSON
with open('vehicles.json') as json_file:
    data = json.load(json_file)
vehicles_df = pd.DataFrame(data).fillna(0).drop(labels=['objectType','country'],axis=1)

# Loading in Solar PV System data JSON
with open('solarpv_systems.json') as json_file:
    data = json.load(json_file)

solar_pv_systems_df = pd.DataFrame(data).fillna(0).drop(labels=['objectType','country'],axis=1)
solar_pv_systems_df

# Loading in Heating Systems data JSON
with open('heating_systems.json') as json_file:
    data = json.load(json_file)

heating_systems_df = pd.DataFrame(data).fillna(0).drop(labels=['objectType','country'],axis=1)
heating_systems_df

# Loading in Battery Storage Systems data JSON
with open('battery_storage_systems.json') as json_file:
    data = json.load(json_file)

battery_storage_systems_df = pd.DataFrame(data).fillna(0).drop(labels=['objectType','country'],axis=1)
# battery_storage_systems_df = battery_storage_systems_df.head(2)


# Loading in EV Home Charger data JSON
with open('ev_chargers.json') as json_file:
    data = json.load(json_file)
ev_chargers_df = pd.DataFrame(data).fillna(0).drop(labels=['objectType','country'],axis=1)

# Loading in Energy Tariffs (Electricity & Gas) data JSON
with open('energy_tariffs.json') as json_file:
    data = json.load(json_file)
energy_tariffs_df = pd.DataFrame(data).fillna(0)
# energy_tariffs_df = energy_tariffs_df.head(2)

# Loading in Solar Irradiance, PV Power & Temperature data JSON
with open('irradiance_temperature_pvpower.json') as json_file:
    data = json.load(json_file)

# Resampling Irradiance, PV Power & Temperature data to half-hourly
# Sourced from EU-PVGIS for crystalline-Si systems (see object for full properties) - https://re.jrc.ec.europa.eu/pvg_tools/en/

location_irradiance_df = pd.DataFrame(data).fillna(0)    
solar_pv_half_hourly_df_list = []
solar_pv_location_df_list = []
for n in range(len(data)):
    location_meta_df = pd.DataFrame(data={'location_id':[data[n]['location_id']],
                                          'location_name':[data[n]['location_name']],
                                          'latitude':[data[n]['inputs']['location']['latitude']],
                                          'longitude':[data[n]['inputs']['location']['longitude']],
                                          'slope':[data[n]['inputs']['mounting_system']['fixed']['slope']['value']],
                                          'azimuth':[data[n]['inputs']['mounting_system']['fixed']['azimuth']['value']],
                                          'pv_system_peak_power_kWp':[data[n]['inputs']['pv_module']['peak_power']]
                                         })
    
    df = pd.DataFrame(data=data[n]['outputs']['hourly'])
    df['datetime'] = pd.to_datetime(df['time'], format='%Y%m%d:%H%M').dt.round('h')
    df.drop(labels=['time','Int','H_sun'], inplace=True, axis=1)
    df.rename(columns={'P':'watts_per_kWp',
                       'G(i)':'global_irrad_Wm-2',
                       'T2m':'temperature_2m_degC'
                       },inplace=True)

    df['datetime'] = pd.to_datetime(df['datetime'])
    df = df.set_index('datetime')
    df = df.resample('30min').interpolate().reset_index()

    df['location_id'] = data[n]['location_id']
    df['day_of_year'] = df['datetime'].dt.dayofyear
#     df['month'] = df['datetime'].dt.month
    df['hour'] = df['datetime'].dt.hour
    df['minute'] = df['datetime'].dt.minute    
    
    solar_pv_location_df_list.append(location_meta_df)
    solar_pv_half_hourly_df_list.append(df)


solar_pv_half_hourly_df = pd.concat(solar_pv_half_hourly_df_list)
solar_pv_location_df = pd.concat(solar_pv_location_df_list)

temperature_daily_df = solar_pv_half_hourly_df.groupby('day_of_year')['temperature_2m_degC'].mean().reset_index()

solar_pv_half_hourly_df.drop(labels=['datetime','temperature_2m_degC'], axis=1, inplace=True)
    

# Estimating Daily Gas Demand based on outdoor temperature and Annual Gas demand
# Using empirical estimator from S.D.Watson et al, https://www.sciencedirect.com/science/article/pii/S0301421518307249

temperature_daily_df = temperature_daily_df.loc[temperature_daily_df['day_of_year']<=365]

temperature_daily_df['daily_gas_demand_kWh_raw'] = 0.
cond1 = (temperature_daily_df['temperature_2m_degC']<14.2)
temperature_daily_df['daily_gas_demand_kWh_raw'].loc[cond1] = (-5.463*temperature_daily_df['temperature_2m_degC'].loc[cond1])+90.55

cond2 = (temperature_daily_df['temperature_2m_degC']>=14.2)
temperature_daily_df['daily_gas_demand_kWh_raw'].loc[cond2] = (-0.988*temperature_daily_df['temperature_2m_degC'].loc[cond2])+26.84

raw_gas_demand_kWh = temperature_daily_df['daily_gas_demand_kWh_raw'].sum()

temperature_daily_df['daily_proportion_of_annual_gas_demand'] = (temperature_daily_df['daily_gas_demand_kWh_raw']/
                                                                     raw_gas_demand_kWh)

print(temperature_daily_df['daily_proportion_of_annual_gas_demand'].sum())

drop_cols = ['daily_gas_demand_kWh_raw']
temperature_daily_df.drop(labels=drop_cols, axis=1, inplace=True)

temperature_daily_df


1.0


/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,day_of_year,temperature_2m_degC,daily_proportion_of_annual_gas_demand
0,1,4.367128,0.004740
1,2,8.180833,0.003259
2,3,6.879271,0.003765
3,4,6.244271,0.004011
4,5,5.881146,0.004152
...,...,...,...
360,361,6.154167,0.004046
361,362,0.567396,0.006215
362,363,1.002917,0.006046
363,364,0.807812,0.006122


In [31]:
# Converting & expanding Energy Tariff JSON into useful dataframes

import itertools

rates_df_list = []
standing_charges_df_list = []
for y in range(len(energy_tariffs_df.index)):
    
    for x in energy_tariffs_df.iloc[y]['tariff_rates']:
        if x['start_settlement_period'] > x['last_settlement_period']:
            settlement_periods = np.arange(x['start_settlement_period'],48,1)
            settlement_periods = np.append(settlement_periods, np.arange(0,x['last_settlement_period']+1,1))
        else:
            settlement_periods = np.arange(x['start_settlement_period'], x['last_settlement_period']+1, 1)
        days_of_week = np.array(x['days_of_week'])
        a = [settlement_periods,
             days_of_week]

        combinations = list(itertools.product(*a))
        rate_df = pd.DataFrame(data=combinations, columns=['settlement_period',
                                                      'day_of_week'])
        rate_df['fuel'] = x['fuel']
        rate_df['period_name'] = x['period_name']    
        rate_df['unit_rate'] = x['unit_rate']
        rate_df['tariff_id'] = energy_tariffs_df.iloc[y]['tariff_id']
        rate_df['tariff_type'] = energy_tariffs_df.iloc[y]['tariff_type']        
        rates_df_list.append(rate_df)

    standing_charge_df = pd.DataFrame(data=energy_tariffs_df.iloc[y]['tariff_standing_charge'])
    standing_charge_df['tariff_id'] = energy_tariffs_df.iloc[y]['tariff_id']
    standing_charges_df_list.append(standing_charge_df)
    

rates_df = pd.concat(rates_df_list)
standing_charges_df = pd.concat(standing_charges_df_list)
print (standing_charges_df)
print (rates_df)

          fuel    cost frequency  tariff_id
0          gas  0.2722     daily          0
1  electricity  0.4448     daily          0
0          gas  0.1437     daily          1
1  electricity  0.2376     daily          1
     settlement_period  day_of_week         fuel period_name  unit_rate  \
0                    0            0  electricity     all-day     0.2763   
1                    0            1  electricity     all-day     0.2763   
2                    0            2  electricity     all-day     0.2763   
3                    0            3  electricity     all-day     0.2763   
4                    0            4  electricity     all-day     0.2763   
..                 ...          ...          ...         ...        ...   
331                 47            2          gas     all-day     0.0600   
332                 47            3          gas     all-day     0.0600   
333                 47            4          gas     all-day     0.0600   
334                 47        

In [32]:
# Pivoting out rates into separate columns for gas, electricity prices, identifying periods of low prices (if any)

rates_df.loc[(rates_df['tariff_id']==3)&
             (rates_df['fuel']=='gas')]

rates_df_pivoted = pd.pivot_table(rates_df, values='unit_rate', index=['settlement_period', 'day_of_week',
                                                                       'tariff_id','tariff_type'],
                    columns=['fuel'], aggfunc=np.sum).reset_index()

rates_df_pivoted.rename(columns={'electricity':'electricity_unit_rate_per_kWh',
                                 'gas':'gas_unit_rate_per_kWh'}, inplace=True)

avg_rates_df = rates_df_pivoted.groupby(['tariff_id','tariff_type'])[['electricity_unit_rate_per_kWh','gas_unit_rate_per_kWh']].mean().reset_index()

avg_rates_df.rename(columns={'electricity_unit_rate_per_kWh':'mean_electricity_unit_rate_per_kWh',
                             'gas_unit_rate_per_kWh':'mean_gas_unit_rate_per_kWh'}, inplace=True)

rates_df_pivoted = pd.merge(rates_df_pivoted, avg_rates_df,
                            on=['tariff_id','tariff_type'])

rates_df_pivoted['electricity_at_or_below_mean_rate'] = False
cond = (rates_df_pivoted['electricity_unit_rate_per_kWh']<rates_df_pivoted['mean_electricity_unit_rate_per_kWh'])
rates_df_pivoted['electricity_at_or_below_mean_rate'].loc[cond] = True

rates_df_pivoted['gas_at_or_below_mean_rate'] = False
cond = (rates_df_pivoted['gas_unit_rate_per_kWh']<rates_df_pivoted['mean_gas_unit_rate_per_kWh'])
rates_df_pivoted['gas_at_or_below_mean_rate'].loc[cond] = True

rates_df_pivoted.drop(labels=['mean_electricity_unit_rate_per_kWh','mean_gas_unit_rate_per_kWh'],
                      axis=1,inplace=True)

print (rates_df_pivoted['settlement_period'].unique())

print ('rates_df Mem Usage:',rates_df.memory_usage().sum()/1e6)
print ('avg_rates_df Mem Usage:',avg_rates_df.memory_usage().sum()/1e6)
print ('rates_df_pivoted Mem Usage:',rates_df_pivoted.memory_usage().sum()/1e6)


standing_charges_df_pivoted = pd.pivot_table(standing_charges_df, values='cost', index=['tariff_id'],
                    columns=['fuel'], aggfunc=np.sum).reset_index()
standing_charges_df_pivoted.index.name = None
# standing_charges_df_pivoted.drop(labels=['fuel'],axis=0, inplace=True)
standing_charges_df_pivoted.rename(columns={'electricity':'electricity_standing_charge_daily',
                                 'gas':'gas_standing_charge_daily'}, inplace=True)

energy_tariffs_df = pd.merge(energy_tariffs_df, standing_charges_df_pivoted, on='tariff_id')
energy_tariffs_df
# standing_charges_df_pivoted
# rates_df_pivoted



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47]
rates_df Mem Usage: 0.086016
avg_rates_df Mem Usage: 0.000192
rates_df_pivoted Mem Usage: 0.038976


/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,tariff_id,tariff_name,supplier_id,supplier_name,tariff_type,tariff_requires_smart_meter,tariff_rates,tariff_standing_charge,tariff_updated_on,electricity_standing_charge_daily,gas_standing_charge_daily
0,0,Flexible Octopus - April 2022 Price Cap,0,Octopus Energy,flat,False,"[{'fuel': 'electricity', 'period_name': 'all-d...","[{'fuel': 'gas', 'cost': 0.2722, 'frequency': ...",2022-04-02T00:00:00,0.4448,0.2722
1,1,Octopus Go - 4H 0030-0430,0,Octopus Energy,tou,True,"[{'fuel': 'electricity', 'period_name': 'off-p...","[{'fuel': 'gas', 'cost': 0.1437, 'frequency': ...",2022-03-03T16:54:00,0.2376,0.1437


In [33]:
# Creating scenarios using cross-products to generate all possible combinations of products, locations & profiles

scenario_df = pd.merge(vehicles_df, battery_storage_systems_df, how='cross')
scenario_df = pd.merge(scenario_df, heating_systems_df, how='cross')
scenario_df = pd.merge(scenario_df, solar_pv_systems_df, how='cross')
scenario_df = pd.merge(scenario_df, ev_chargers_df, how='cross')
scenario_df = pd.merge(scenario_df, energy_tariffs_df[['tariff_id','tariff_name','tariff_requires_smart_meter',
                                                       'electricity_standing_charge_daily',
                                                       'gas_standing_charge_daily']], how='cross')

scenario_df = pd.merge(scenario_df, location_irradiance_df[['location_id','location_name']], how='cross')

scenario_df = pd.merge(scenario_df, energy_consumption_levels_df, how='cross')

scenario_df = pd.merge(scenario_df, profile_grouped_df, how='cross')

scenario_df['electricity_profile_multiplier'] = (scenario_df['annual_electricity_consumption_kWh']/
                                                 scenario_df['electricity_demand_normalised'])

scenario_df.drop(labels=['electricity_demand_normalised','heating_system_coefficient_of_performance'], axis=1, inplace=True)

scenario_df['scenario_id'] = [n for n in range(len(scenario_df.index))]


scenario_df['electricity_demand_ev_daily_Wh'] = scenario_df['daily_miles_driven']*scenario_df['vehicle_wh_per_mile']


print ('Reading in Data & Generating the scenarios took:',time.time()-ttt,'seconds')
print ('scenario_df Mem Usage:',scenario_df.memory_usage().sum()/1e6)
print (scenario_df.iloc[0])

# non_gas_heating_cond = (scenario_df['heating_system_fuel_type']!='gas')
# scenario_df = scenario_df

scenario_df

Reading in Data & Generating the scenarios took: 0.5945870876312256 seconds
scenario_df Mem Usage: 0.960768
vehicle_id                                                                        0
vehicle_name                                              Typical Petrol/Diesel Car
vehicle_type                                                            ICE Vehicle
vehicle_fuel_type                                                          gasoline
vehicle_miles_per_gallon                                                       40.0
vehicle_cost                                                                  37000
vehicle_currency                                                                GBP
vehicle_wh_per_mile                                                             0.0
vehicle_max_charge_rate_watts                                                   0.0
vehicle_battery_capacity_Wh                                                     0.0
battery_storage_id                                  

,vehicle_id,vehicle_name,vehicle_type,vehicle_fuel_type,vehicle_miles_per_gallon,vehicle_cost,vehicle_currency,vehicle_wh_per_mile,vehicle_max_charge_rate_watts,vehicle_battery_capacity_Wh,...,gas_standing_charge_daily,location_id,location_name,annual_electricity_consumption_kWh,annual_gas_consumption_kWh,daily_miles_driven,profile_id,electricity_profile_multiplier,scenario_id,electricity_demand_ev_daily_Wh
0,0,Typical Petrol/Diesel Car,ICE Vehicle,gasoline,40.0,37000,GBP,0.0,0.0,0.0,...,0.2722,0,Thames,2200,10000,0,0,6.027397,0,0.0
1,0,Typical Petrol/Diesel Car,ICE Vehicle,gasoline,40.0,37000,GBP,0.0,0.0,0.0,...,0.2722,0,Thames,2200,10000,20,0,6.027397,1,0.0
2,0,Typical Petrol/Diesel Car,ICE Vehicle,gasoline,40.0,37000,GBP,0.0,0.0,0.0,...,0.2722,0,Thames,2200,10000,50,0,6.027397,2,0.0
3,0,Typical Petrol/Diesel Car,ICE Vehicle,gasoline,40.0,37000,GBP,0.0,0.0,0.0,...,0.2722,0,Thames,2200,10000,100,0,6.027397,3,0.0
4,0,Typical Petrol/Diesel Car,ICE Vehicle,gasoline,40.0,37000,GBP,0.0,0.0,0.0,...,0.2722,0,Thames,2200,12000,0,0,6.027397,4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,1,Typical EV,Electric Vehicle,electricity,0.0,50000,GBP,250.0,250000.0,70000.0,...,0.1437,0,Thames,3600,12000,100,0,9.863014,2299,25000.0
2300,1,Typical EV,Electric Vehicle,electricity,0.0,50000,GBP,250.0,250000.0,70000.0,...,0.1437,0,Thames,3600,14000,0,0,9.863014,2300,0.0
2301,1,Typical EV,Electric Vehicle,electricity,0.0,50000,GBP,250.0,250000.0,70000.0,...,0.1437,0,Thames,3600,14000,20,0,9.863014,2301,5000.0
2302,1,Typical EV,Electric Vehicle,electricity,0.0,50000,GBP,250.0,250000.0,70000.0,...,0.1437,0,Thames,3600,14000,50,0,9.863014,2302,12500.0


In [34]:
# Chunking scenarios into blocks of 200 to run at a time - to avoid out of em

n = 200  #chunk row size
n_chunks = math.ceil(len(scenario_df.index)/n)
scenario_df_chunked = [scenario_df[i*n:(i+1)*n] for i in range(n_chunks)]


In [35]:
# For a given chunk of scenarios, we'd like to generate a dataframe that contains all necessary 
# info to perform half-hourly simulations of performance.

# We are avoiding doing this for ALL scenarios (there may be 10K+, depending on number of input products),
# to avoid out-of-memory problems (testing on 2018 MBP with 8GB RAM)

def generate_half_hourly_simulation_df(scenario_df, temperature_daily_df,
                                       baseload_profile_df, rates_df_pivoted,
                                       solar_pv_half_hourly_df, 
                                       ):
    ttt = time.time()
    hh = pd.date_range("2021-01-01T00:00:00", "2021-12-31T23:30:00", freq="30min")
    half_hourly_df = pd.DataFrame(data={'datetime':pd.date_range("2021-01-01T00:00:00", 
                                                                 "2021-12-31T23:30:00", 
                                                                 freq="30min")}
                                 )

    half_hourly_df['day_of_year'] = half_hourly_df['datetime'].dt.dayofyear
    half_hourly_df['day_of_week'] = half_hourly_df['datetime'].dt.dayofweek
    half_hourly_df['month'] = half_hourly_df['datetime'].dt.month
    half_hourly_df['hour'] = half_hourly_df['datetime'].dt.hour
    half_hourly_df['minute'] = half_hourly_df['datetime'].dt.minute
    half_hourly_df['settlement_period'] = ((half_hourly_df['hour']*2)+(half_hourly_df['minute']/30.)).astype(int)

    half_hourly_baseload_profile_df = pd.merge(scenario_df, half_hourly_df, 
                                               how='cross')    

    ttt = time.time()
    half_hourly_baseload_profile_df = pd.merge(half_hourly_baseload_profile_df, temperature_daily_df,
                                               on='day_of_year',
                                               how='inner')

    half_hourly_baseload_profile_df = pd.merge(half_hourly_baseload_profile_df, baseload_profile_df,
                                               on=['profile_id','hour','minute'], how='inner')

    print ('Merging with baseload_profile_df:',time.time()-ttt,'seconds')

    half_hourly_baseload_profile_df['electricity_demand_baseload_Wh'] = (1000*
                                                                         half_hourly_baseload_profile_df['electricity_profile_multiplier']*
                                                                         half_hourly_baseload_profile_df['electricity_demand_normalised']
                                                                        )

    half_hourly_baseload_profile_df['heat_demand_normalised_Wh'] = 1000*half_hourly_baseload_profile_df['annual_gas_consumption_kWh']*half_hourly_baseload_profile_df['daily_proportion_of_annual_gas_demand']/48.
    # half_hourly_baseload_profile_df['heat_demand_normalised_Wh'] = 1000*half_hourly_baseload_profile_df['annual_gas_consumption_kWh']/(8760*2)


    print ('Generating Demand Based on Profile Shape:',time.time()-ttt,'seconds')

    print ('half_hourly_baseload_profile_df Mem Usage:',half_hourly_baseload_profile_df.memory_usage().sum()/1e9,'GB')
    
    ttt = time.time()

    half_hourly_baseload_profile_df = pd.merge(half_hourly_baseload_profile_df, rates_df_pivoted,
                                               on=['settlement_period','day_of_week','tariff_id'],
                                               how='left'
                                              )

    print ('half_hourly_baseload_profile_df Mem Usage:',half_hourly_baseload_profile_df.memory_usage().sum()/1e9,'GB')
    print ('Adding Rates to half-hourly DF:',time.time()-ttt)


    half_hourly_baseload_profile_df    
    
    ttt = time.time()

    cols = ['scenario_id','heating_system_id','heating_system_name','heating_system_fuel_type',
            'heating_system_efficiency','heat_demand_normalised_Wh']


    half_hourly_baseload_profile_df['electricity_demand_heatpump_Wh'] = 0.
    cond = (half_hourly_baseload_profile_df['heating_system_fuel_type']=='electricity')
    half_hourly_baseload_profile_df['electricity_demand_heatpump_Wh'].loc[cond] = (half_hourly_baseload_profile_df['heat_demand_normalised_Wh'].loc[cond]/
                                                                                   half_hourly_baseload_profile_df['heating_system_efficiency'].loc[cond])

    half_hourly_baseload_profile_df['gas_demand_Wh'] = 0.
    cond = (half_hourly_baseload_profile_df['heating_system_fuel_type']=='gas')
    half_hourly_baseload_profile_df['gas_demand_Wh'].loc[cond] = (half_hourly_baseload_profile_df['heat_demand_normalised_Wh'].loc[cond]/
                                                                                   half_hourly_baseload_profile_df['heating_system_efficiency'].loc[cond])
    # half_hourly_baseload_profile_df[cols]
    # half_hourly_baseload_profile_df.groupby('scenario_id')['heat_demand_normalised_Wh'].sum()

    print ('half_hourly_baseload_profile_df Mem Usage:',half_hourly_baseload_profile_df.memory_usage().sum()/1e9,'GB')
    print ('Generating Elec & Natural Gas Demand from heating:',time.time()-ttt,'seconds')

    ttt = time.time()
    half_hourly_baseload_profile_df = pd.merge(half_hourly_baseload_profile_df,
                                               solar_pv_half_hourly_df,
                                               on=['location_id','day_of_year','hour','minute'],
                                               how='left').fillna(0)

    drop_cols = ['WS10m','electricity_demand_normalised','electricity_profile_multiplier','electricity_demand_ev_daily_Wh',
                'day_of_year','day_of_week','month','hour','minute',
                'ev_charger_cost','ev_charger_currency','solar_pv_cost','solar_pv_currency',
                'heating_system_cost','heating_system_currency',
                'battery_storage_cost','battery_storage_currency',
                'vehicle_cost','vehicle_currency','vehicle_type']
    half_hourly_baseload_profile_df.drop(labels=drop_cols, axis=1, inplace=True)


    print ('half_hourly_baseload_profile_df Mem Usage:',half_hourly_baseload_profile_df.memory_usage().sum()/1e9,'GB')
    print ('Merging with solar_pv_half_hourly_df:',time.time()-ttt,'seconds')

    # Consider interpolating here - sometimes, there is a half-hour missing!
    # df = df.set_index('datetime')
    # df = df.resample('30min').interpolate().reset_index()

    half_hourly_baseload_profile_df    
    half_hourly_baseload_profile_df.loc[half_hourly_baseload_profile_df['solar_pv_system_size_Wp']>0]

    half_hourly_baseload_profile_df['solar_pv_generation_Wh'] = (0.5*half_hourly_baseload_profile_df['watts_per_kWp']*
                                                                 half_hourly_baseload_profile_df['solar_pv_system_size_Wp']/1000.
                                                                )

    return half_hourly_baseload_profile_df
    

In [36]:
def simulate_half_hourly_performance(scenario_df, half_hourly_baseload_profile_df):
    scenario_id_list = []
    ds_list = []
    baseload_elec_demand_list = []
    grid_elec_import_list = []
    grid_elec_export_list = []
    grid_elec_import_price_per_Wh_list = []
    battery_generation_Wh_list = []
    battery_charging_demand_Wh_list = []
    battery_energy_stored_energy_Wh_beginning_of_period_list = []
    battery_energy_stored_energy_Wh_end_of_period_list = []
    ev_energy_stored_energy_Wh_beginning_of_period_list = []
    ev_energy_stored_energy_Wh_end_of_period_list = []
    ev_charging_demand_Wh_list = []
    electricity_unit_rate_per_kWh_list = []
    gas_unit_rate_per_kWh_list = []
    electricity_demand_baseload_Wh_list = []
    electricity_demand_heatpump_Wh_list = []
    gas_demand_Wh_list = []
    heat_demand_normalised_Wh_list = []
    solar_pv_generation_Wh_list = []
    
    n_dates = len(half_hourly_baseload_profile_df['datetime'].unique())
    n_scenarios = len(half_hourly_baseload_profile_df['scenario_id'].unique())
#     print (n_scenarios)

    battery_energy_stored_energy_Wh_beginning_of_period = np.zeros(n_scenarios)
    battery_energy_stored_energy_Wh_end_of_period = np.zeros(n_scenarios)
    battery_energy_max_capacity_Wh = scenario_df['battery_storage_capacity_Wh'].values


    battery_storage_charging_efficiency = scenario_df['battery_storage_charging_efficiency'].values
    battery_storage_max_discharge_rate_watts = scenario_df['battery_storage_max_discharge_rate_watts'].values
    battery_storage_max_charge_rate_watts = scenario_df['battery_storage_max_charge_rate_watts'].values

    ev_max_charge_rate_watts = scenario_df['ev_charger_max_charge_rate_watts'].values
    ev_max_input_Wh = ev_max_charge_rate_watts*0.5
    ev_energy_max_capacity_Wh = scenario_df['electricity_demand_ev_daily_Wh'].values
    ev_energy_stored_energy_Wh_beginning_of_period = np.zeros(n_scenarios)
    ev_energy_stored_energy_Wh_end_of_period = np.zeros(n_scenarios)

    t_track = time.time()
    for t in ds:
        battery_energy_stored_energy_Wh_beginning_of_period = battery_energy_stored_energy_Wh_end_of_period
        ev_energy_stored_energy_Wh_beginning_of_period = ev_energy_stored_energy_Wh_end_of_period    

        battery_energy_until_full_Wh_start_of_period = battery_energy_max_capacity_Wh - battery_energy_stored_energy_Wh_beginning_of_period    
        ev_energy_until_full_Wh_start_of_period = ev_energy_max_capacity_Wh - ev_energy_stored_energy_Wh_beginning_of_period

        current_timestep_cond = (half_hourly_baseload_profile_df['datetime'] == t)
        current_timestep_df = half_hourly_baseload_profile_df.loc[current_timestep_cond].copy()    

        heatpump_elec_loss_Wh = np.array([0 for nn in range(len(current_timestep_df.index))])
        ev_charger_elec_loss_Wh = np.array([0 for nn in range(len(current_timestep_df.index))])

        below_avg_elec_price = current_timestep_df['electricity_at_or_below_mean_rate'].values
        electricity_unit_rate_per_kWh = current_timestep_df['electricity_unit_rate_per_kWh'].values
        gas_unit_rate_per_kWh = current_timestep_df['gas_unit_rate_per_kWh'].values
        scenario_id = current_timestep_df['scenario_id'].values
        baseload_elec_demand_Wh = current_timestep_df['electricity_demand_baseload_Wh'].values
        heatpump_elec_demand_Wh = current_timestep_df['electricity_demand_heatpump_Wh'].values
        ev_charging_demand_Wh = np.zeros(n_scenarios)
        pv_generation_Wh = current_timestep_df['solar_pv_generation_Wh'].values
        gas_demand_Wh = current_timestep_df['gas_demand_Wh'].values
        heat_demand_normalised_Wh = current_timestep_df['heat_demand_normalised_Wh'].values        

        settlement_period = current_timestep_df['settlement_period'].values
        tariff_type = current_timestep_df['tariff_type'].values

    #     Reset EV energy stored to zero every day at 18:00, or HH settlement period 36
        ev_reset_condition = np.where(settlement_period == 36)
        ev_energy_stored_energy_Wh_beginning_of_period[ev_reset_condition] = 0.

    #     Determining when the EV should be charging
#         Where Time-of-Use tariffs are active, charge when it's cheap
#         Where Time-of-Use tariffs are active, charge after it's cheap in order to top up range
#         Where flat rate tariffs are active, charge any time after 6pm (when people get home, roughly)
        ev_charging_condition = np.where(
            (
                (tariff_type == 'tou')&
                (ev_energy_stored_energy_Wh_beginning_of_period < ev_energy_max_capacity_Wh)&
                (below_avg_elec_price == True)
            ) | 
            (
                (tariff_type == 'tou')&
                (ev_energy_stored_energy_Wh_beginning_of_period < ev_energy_max_capacity_Wh)&
                (below_avg_elec_price == False) & 
                (settlement_period >= 4)
            )
              |
            (
                (tariff_type == 'flat')&
                (ev_energy_stored_energy_Wh_beginning_of_period < ev_energy_max_capacity_Wh)
            )
        )

        ev_charging_demand_Wh[ev_charging_condition] = np.minimum(ev_max_input_Wh[ev_charging_condition],
                                                                  ev_energy_until_full_Wh_start_of_period[ev_charging_condition]
                                                                 )

        ev_energy_stored_energy_Wh_end_of_period = (ev_energy_stored_energy_Wh_beginning_of_period +
                                                    ev_charging_demand_Wh)


        non_battery_demand_Wh = ev_charging_demand_Wh + baseload_elec_demand_Wh + heatpump_elec_demand_Wh    

        non_battery_demand_net_of_pv_gen_Wh = non_battery_demand_Wh - pv_generation_Wh

        non_battery_demand_after_pv_gen_Wh = np.maximum(non_battery_demand_net_of_pv_gen_Wh, 0)

        pv_excess_condition = np.where(non_battery_demand_net_of_pv_gen_Wh < 0.)
        pv_satisfy_non_battery_demand_Wh = np.minimum(non_battery_demand_Wh, pv_generation_Wh)

        pv_excess_generation_Wh = pv_generation_Wh - pv_satisfy_non_battery_demand_Wh
        grid_elec_import_Wh = np.zeros(n_scenarios)




        battery_max_input_Wh = battery_storage_max_charge_rate_watts*0.5

        battery_generation_Wh = np.zeros(n_scenarios)

        battery_charging_demand_Wh = np.zeros(n_scenarios)


    #     Deciding on when to charge and discharge the battery
    #     If PV has excess, and electricity is more expensive than average, 
    #     we should only charge with the excess PV electricity, and not import any electricity from the grid

        battery_charge_condition_1 = np.where((battery_energy_stored_energy_Wh_beginning_of_period < battery_energy_max_capacity_Wh)&
                                              (pv_excess_generation_Wh > 0.)&
                                              (below_avg_elec_price == False)
                                             )
    #     Input battery_storage_efficiency here - basically, to get 1kWh of energy, you need to out in 1/efficiency = 1.05 kWh
        battery_charging_demand_Wh[battery_charge_condition_1] = np.minimum(pv_excess_generation_Wh[battery_charge_condition_1],
                                                                            battery_energy_until_full_Wh_start_of_period[battery_charge_condition_1],
                                                                            battery_max_input_Wh[battery_charge_condition_1])


    #     If PV has excess, and electricity is less expensive than average, 
    #     Input battery_storage_efficiency here - basically, to get 1kWh of energy, you need to out in 1/efficiency = 1.05 kWh
        battery_charge_condition_2 = np.where((battery_energy_stored_energy_Wh_beginning_of_period < battery_energy_max_capacity_Wh)&
                                              (pv_excess_generation_Wh > 0.)&
                                              (below_avg_elec_price == True)
                                             )
        battery_charging_demand_Wh[battery_charge_condition_2] = np.minimum(battery_energy_until_full_Wh_start_of_period[battery_charge_condition_2],
                                                                             battery_max_input_Wh[battery_charge_condition_2])

        grid_elec_import_Wh[battery_charge_condition_2] = battery_charging_demand_Wh[battery_charge_condition_2] - pv_excess_generation_Wh[battery_charge_condition_2]



    #     If PV does not have excess, and electricity is LESS expensive than average, 
    #     Input battery_storage_efficiency here - basically, to get 1kWh of energy, you need to out in 1/efficiency = 1.05 kWh    
        battery_charge_condition_3 = np.where((battery_energy_stored_energy_Wh_beginning_of_period < battery_energy_max_capacity_Wh)&
                                              (pv_excess_generation_Wh <= 0.)&
                                              (below_avg_elec_price == True)
                                             )
        battery_charging_demand_Wh[battery_charge_condition_3] = np.minimum(battery_energy_until_full_Wh_start_of_period[battery_charge_condition_3],
                                                                            battery_max_input_Wh[battery_charge_condition_3],
                                                                            )    

        grid_elec_import_Wh[battery_charge_condition_3] = battery_charging_demand_Wh[battery_charge_condition_3]

    #     If PV does not have excess, and electricity is MORE expensive than average, we should not charge.
    #     Instead, we should discharge the battery

    #     If electricity is more expensive than average...    
    #     battery_discharge_condition_1 = np.where((pv_satisfy_non_battery_demand_Wh <= 0.) & 
    #                                              (battery_energy_stored_energy_Wh_beginning_of_period > 0.) & 
    #                                              (below_avg_elec_price == False)
    #                                             )

        battery_discharge_condition_1 = np.where((pv_excess_generation_Wh <= 0.) & 
                                                 (battery_energy_stored_energy_Wh_beginning_of_period > 0.) & 
                                                 (below_avg_elec_price == False)
                                                )    

    #     Discharge the kWh demand, or the available capacity inside the battery, whichever is low
        battery_generation_Wh[battery_discharge_condition_1] = np.minimum(non_battery_demand_net_of_pv_gen_Wh[battery_discharge_condition_1],
                                                                     battery_energy_stored_energy_Wh_beginning_of_period[battery_discharge_condition_1]
                                                                    )

    #     Or, we should discharge the battery where we know there's going to be sufficient solar power to charge 
    #     it up tomorrow, even if power is cheap right now.  This is WIP!

    #     battery_bottom_reserve_kWh = np.array(battery_energy_max_capacity_kWh) - (0.5*np.array([merged_df['P_following_day'].values[t]]))

    #     battery_bottom_reserve_kWh = [0. for s in scenario_id]

    #     battery_discharge_condition_2 = np.where((pv_satisfy_non_battery_demand_kWh <= 0.) & 
    #                                              (battery_energy_stored_energy_kWh_beginning_of_period > battery_bottom_reserve_kWh) & 
    #                                              (below_avg_elec_price == True)
    #                                             )
    #     battery_generation_kWh[battery_discharge_condition_2] = np.minimum(non_battery_demand_net_of_pv_gen_kWh[battery_discharge_condition_2],
    #                                                              battery_energy_stored_energy_kWh_beginning_of_period[battery_discharge_condition_2]
    #                                                             )


        pv_satisfy_battery_demand_Wh = np.minimum(battery_charging_demand_Wh, pv_excess_generation_Wh)

        pv_export_Wh = pv_generation_Wh - pv_satisfy_non_battery_demand_Wh - pv_satisfy_battery_demand_Wh

        grid_elec_import_Wh = (non_battery_demand_Wh -
                                pv_satisfy_non_battery_demand_Wh - 
                                battery_generation_Wh +
                                battery_charging_demand_Wh - 
                                pv_satisfy_battery_demand_Wh)

        grid_elec_export_Wh = (pv_satisfy_non_battery_demand_Wh - 
                                non_battery_demand_Wh + 
                                pv_satisfy_battery_demand_Wh +
                                pv_export_Wh - 
                                battery_charging_demand_Wh +
                                battery_generation_Wh + 
                                grid_elec_import_Wh
                               )



    #     Charge the battery using PV excess energy, but don't charge using grid, as the grid is expensive.
    #     Of the kWh to full battery, and the available PV excess energy, pick the lower value

    #     Charge the battery using PV excess energy, as well as charge using grid, as the grid is cheap.
    #     Of the kWh to full battery, and the kWh deliverable by operating at max charge rate, pick the lower value

        battery_energy_stored_energy_Wh_end_of_period = (battery_energy_stored_energy_Wh_beginning_of_period +
                                                          battery_charging_demand_Wh - 
                                                          battery_generation_Wh
                                                         )

    #     If PV does not have excess, and electricity is less expensive than average, 


        ds_list.append([t for s in scenario_id])
        scenario_id_list.append(scenario_id)
        baseload_elec_demand_list.append(baseload_elec_demand_Wh)
        grid_elec_import_list.append(grid_elec_import_Wh)
        grid_elec_export_list.append(grid_elec_export_Wh)
        battery_generation_Wh_list.append(battery_generation_Wh)
        battery_charging_demand_Wh_list.append(battery_charging_demand_Wh)
        battery_energy_stored_energy_Wh_beginning_of_period_list.append(battery_energy_stored_energy_Wh_beginning_of_period)
        battery_energy_stored_energy_Wh_end_of_period_list.append(battery_energy_stored_energy_Wh_end_of_period)    
        ev_energy_stored_energy_Wh_beginning_of_period_list.append(ev_energy_stored_energy_Wh_beginning_of_period)
        ev_energy_stored_energy_Wh_end_of_period_list.append(ev_energy_stored_energy_Wh_end_of_period)
        ev_charging_demand_Wh_list.append(ev_charging_demand_Wh)
        electricity_unit_rate_per_kWh_list.append(electricity_unit_rate_per_kWh)
        gas_unit_rate_per_kWh_list.append(gas_unit_rate_per_kWh)
        electricity_demand_baseload_Wh_list.append(baseload_elec_demand_Wh)
        electricity_demand_heatpump_Wh_list.append(heatpump_elec_demand_Wh)
        gas_demand_Wh_list.append(gas_demand_Wh)
        heat_demand_normalised_Wh_list.append(heat_demand_normalised_Wh)       
        solar_pv_generation_Wh_list.append(pv_generation_Wh)
        
#     Turning this into a dataframe for return...
    results_df = pd.DataFrame(data={'datetime':np.concatenate(ds_list),
                                    'scenario_id':np.concatenate(scenario_id_list),
                                    'grid_elec_import_Wh':np.concatenate(grid_elec_import_list),
                                    'grid_elec_export_Wh':np.concatenate(grid_elec_export_list),
                                    'battery_generation_Wh':np.concatenate(battery_generation_Wh_list),
                                    'battery_charging_demand_Wh':np.concatenate(battery_charging_demand_Wh_list),
                                    'battery_energy_stored_energy_Wh_beginning_of_period':np.concatenate(battery_energy_stored_energy_Wh_beginning_of_period_list),
                                    'battery_energy_stored_energy_Wh_end_of_period':np.concatenate(battery_energy_stored_energy_Wh_end_of_period_list),
                                    'ev_energy_stored_energy_Wh_beginning_of_period':np.concatenate(ev_energy_stored_energy_Wh_beginning_of_period_list),
                                    'ev_energy_stored_energy_Wh_end_of_period':np.concatenate(ev_energy_stored_energy_Wh_end_of_period_list),
                                    'ev_charging_demand_Wh':np.concatenate(ev_charging_demand_Wh_list),
                                    'electricity_unit_rate_per_kWh':np.concatenate(electricity_unit_rate_per_kWh_list),
                                    'gas_unit_rate_per_kWh':np.concatenate(gas_unit_rate_per_kWh_list),
                                    'electricity_demand_baseload_Wh':np.concatenate(electricity_demand_baseload_Wh_list),
                                    'electricity_demand_heatpump_Wh':np.concatenate(electricity_demand_heatpump_Wh_list),
                                    'gas_demand_Wh':np.concatenate(gas_demand_Wh_list),
                                    'heat_demand_normalised_Wh':np.concatenate(heat_demand_normalised_Wh_list),
                                    'solar_pv_generation_Wh':np.concatenate(solar_pv_generation_Wh_list)
                                    }
                             )   
    print ('Time to run half-hourly simulation:',time.time()-t_track,'seconds')
    return results_df


In [37]:

results_df_list = []
for chunk_number, scenario_df_chunk in enumerate(scenario_df_chunked):

    half_hourly_baseload_profile_df = generate_half_hourly_simulation_df(scenario_df_chunk, temperature_daily_df,
                                                                         baseload_profile_df, rates_df_pivoted,
                                                                         solar_pv_half_hourly_df
                                                                        )
    
    results_df = simulate_half_hourly_performance(scenario_df_chunk, 
                                                  half_hourly_baseload_profile_df)
    results_df_list.append(results_df)
    print (chunk_number, scenario_df_chunk['scenario_id'].min(), scenario_df_chunk['scenario_id'].max(),
          results_df.head())
    
results_df_full = pd.concat(results_df_list)

Merging with baseload_profile_df: 27.891368865966797 seconds
Generating Demand Based on Profile Shape: 27.958470821380615 seconds
half_hourly_baseload_profile_df Mem Usage: 1.825584 GB
half_hourly_baseload_profile_df Mem Usage: 1.916688 GB
Adding Rates to half-hourly DF: 3.201772928237915


/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


half_hourly_baseload_profile_df Mem Usage: 1.972752 GB
Generating Elec & Natural Gas Demand from heating: 0.5274009704589844 seconds
half_hourly_baseload_profile_df Mem Usage: 1.496208 GB
Merging with solar_pv_half_hourly_df: 34.07185220718384 seconds
Time to run half-hourly simulation: 146.6428439617157 seconds
0 0 199     datetime  scenario_id  grid_elec_import_Wh  grid_elec_export_Wh  \
0 2021-01-01            0           109.160197                  0.0   
1 2021-01-01            1           109.160197                  0.0   
2 2021-01-01            2           109.160197                  0.0   
3 2021-01-01            3           109.160197                  0.0   
4 2021-01-01            4           109.160197                  0.0   

   battery_generation_Wh  battery_charging_demand_Wh  \
0                    0.0                         0.0   
1                    0.0                         0.0   
2                    0.0                         0.0   
3                    0.0   

/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


half_hourly_baseload_profile_df Mem Usage: 1.972752 GB
Generating Elec & Natural Gas Demand from heating: 0.7076809406280518 seconds
half_hourly_baseload_profile_df Mem Usage: 1.496208 GB
Merging with solar_pv_half_hourly_df: 34.97392296791077 seconds
Time to run half-hourly simulation: 147.17777276039124 seconds
1 200 399     datetime  scenario_id  grid_elec_import_Wh  grid_elec_export_Wh  \
0 2021-01-01          200           143.892988                  0.0   
1 2021-01-01          201           143.892988                  0.0   
2 2021-01-01          202           143.892988                  0.0   
3 2021-01-01          203           143.892988                  0.0   
4 2021-01-01          204           178.625778                  0.0   

   battery_generation_Wh  battery_charging_demand_Wh  \
0                    0.0                         0.0   
1                    0.0                         0.0   
2                    0.0                         0.0   
3                    0.0

...

half_hourly_baseload_profile_df Mem Usage: 1.972752 GB
Generating Elec & Natural Gas Demand from heating: 0.5741229057312012 seconds
half_hourly_baseload_profile_df Mem Usage: 1.496208 GB
Merging with solar_pv_half_hourly_df: 37.560237884521484 seconds
Time to run half-hourly simulation: 149.2628879547119 seconds
9 1800 1999     datetime  scenario_id  grid_elec_import_Wh  grid_elec_export_Wh  \
0 2021-01-01         1800           109.160197                  0.0   
1 2021-01-01         1801          3809.160197                  0.0   
2 2021-01-01         1802          3809.160197                  0.0   
3 2021-01-01         1803          3809.160197                  0.0   
4 2021-01-01         1804           109.160197                  0.0   

   battery_generation_Wh  battery_charging_demand_Wh  \
0                    0.0                         0.0   
1                    0.0                         0.0   
2                    0.0                         0.0   
3                    0

/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


half_hourly_baseload_profile_df Mem Usage: 1.972752 GB
Generating Elec & Natural Gas Demand from heating: 0.5997140407562256 seconds
half_hourly_baseload_profile_df Mem Usage: 1.496208 GB
Merging with solar_pv_half_hourly_df: 38.59074926376343 seconds
Time to run half-hourly simulation: 148.25066542625427 seconds
10 2000 2199     datetime  scenario_id  grid_elec_import_Wh  grid_elec_export_Wh  \
0 2021-01-01         2000           143.892988                  0.0   
1 2021-01-01         2001           143.892988                  0.0   
2 2021-01-01         2002           143.892988                  0.0   
3 2021-01-01         2003           143.892988                  0.0   
4 2021-01-01         2004           178.625778                  0.0   

   battery_generation_Wh  battery_charging_demand_Wh  \
0                    0.0                         0.0   
1                    0.0                         0.0   
2                    0.0                         0.0   
3                    

/Users/alvin/.pyenv/versions/bdt-backend-env/lib/python3.8/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


half_hourly_baseload_profile_df Mem Usage: 1.02583104 GB
Generating Elec & Natural Gas Demand from heating: 0.2764279842376709 seconds
half_hourly_baseload_profile_df Mem Usage: 0.77802816 GB
Merging with solar_pv_half_hourly_df: 16.174567222595215 seconds
Time to run half-hourly simulation: 82.41779518127441 seconds
11 2200 2303     datetime  scenario_id  grid_elec_import_Wh  grid_elec_export_Wh  \
0 2021-01-01         2200           563.178339                  0.0   
1 2021-01-01         2201           563.178339                  0.0   
2 2021-01-01         2202           563.178339                  0.0   
3 2021-01-01         2203           563.178339                  0.0   
4 2021-01-01         2204           638.848030                  0.0   

   battery_generation_Wh  battery_charging_demand_Wh  \
0                    0.0                         0.0   
1                    0.0                         0.0   
2                    0.0                         0.0   
3                

In [38]:
results_df_full

,datetime,scenario_id,grid_elec_import_Wh,grid_elec_export_Wh,battery_generation_Wh,battery_charging_demand_Wh,battery_energy_stored_energy_Wh_beginning_of_period,battery_energy_stored_energy_Wh_end_of_period,ev_energy_stored_energy_Wh_beginning_of_period,ev_energy_stored_energy_Wh_end_of_period,ev_charging_demand_Wh,electricity_unit_rate_per_kWh,gas_unit_rate_per_kWh,electricity_demand_baseload_Wh,electricity_demand_heatpump_Wh,gas_demand_Wh,heat_demand_normalised_Wh,solar_pv_generation_Wh
0,2021-01-01 00:00:00,0,109.160197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2763,0.0728,109.160197,0.000000,987.489458,987.489458,0.0
1,2021-01-01 00:00:00,1,109.160197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2763,0.0728,109.160197,0.000000,987.489458,987.489458,0.0
2,2021-01-01 00:00:00,2,109.160197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2763,0.0728,109.160197,0.000000,987.489458,987.489458,0.0
3,2021-01-01 00:00:00,3,109.160197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2763,0.0728,109.160197,0.000000,987.489458,987.489458,0.0
4,2021-01-01 00:00:00,4,109.160197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2763,0.0728,109.160197,0.000000,1184.987350,1184.987350,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1822075,2021-12-31 23:30:00,2299,793.345092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.0600,212.437086,580.908007,0.000000,1516.169897,0.0
1822076,2021-12-31 23:30:00,2300,890.163093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.0600,212.437086,677.726008,0.000000,1768.864880,0.0
1822077,2021-12-31 23:30:00,2301,890.163093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.0600,212.437086,677.726008,0.000000,1768.864880,0.0
1822078,2021-12-31 23:30:00,2302,890.163093,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.0600,212.437086,677.726008,0.000000,1768.864880,0.0


In [39]:
# Checking a specific scenario:

results_df_full.loc[results_df_full['scenario_id']==1727]

,datetime,scenario_id,grid_elec_import_Wh,grid_elec_export_Wh,battery_generation_Wh,battery_charging_demand_Wh,battery_energy_stored_energy_Wh_beginning_of_period,battery_energy_stored_energy_Wh_end_of_period,ev_energy_stored_energy_Wh_beginning_of_period,ev_energy_stored_energy_Wh_end_of_period,ev_charging_demand_Wh,electricity_unit_rate_per_kWh,gas_unit_rate_per_kWh,electricity_demand_baseload_Wh,electricity_demand_heatpump_Wh,gas_demand_Wh,heat_demand_normalised_Wh,solar_pv_generation_Wh
127,2021-01-01 00:00:00,1727,708.313610,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.06,178.625778,529.687832,0.0,1382.485242,0.0
327,2021-01-01 00:30:00,1727,4384.390515,0.0,0.0,0.0,0.0,0.0,0.0,3700.0,3700.0,0.0750,0.06,154.702682,529.687832,0.0,1382.485242,0.0
527,2021-01-01 01:00:00,1727,4362.700242,0.0,0.0,0.0,0.0,0.0,3700.0,7400.0,3700.0,0.0750,0.06,133.012409,529.687832,0.0,1382.485242,0.0
727,2021-01-01 01:30:00,1727,4346.113562,0.0,0.0,0.0,0.0,0.0,7400.0,11100.0,3700.0,0.0750,0.06,116.425730,529.687832,0.0,1382.485242,0.0
927,2021-01-01 02:00:00,1727,4336.863299,0.0,0.0,0.0,0.0,0.0,11100.0,14800.0,3700.0,0.0750,0.06,107.175467,529.687832,0.0,1382.485242,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3503127,2021-12-31 21:30:00,1727,973.734439,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.06,296.008432,677.726008,0.0,1768.864880,0.0
3503327,2021-12-31 22:00:00,1727,963.846227,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.06,286.120219,677.726008,0.0,1768.864880,0.0
3503527,2021-12-31 22:30:00,1727,952.363141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.06,274.637133,677.726008,0.0,1768.864880,0.0
3503727,2021-12-31 23:00:00,1727,927.802096,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.3023,0.06,250.076089,677.726008,0.0,1768.864880,0.0


In [40]:


print ('Pre-merge Results Mem Usage:',results_df_full.memory_usage().sum()/1e6)

# print (results_df_full)
# print (results_df_full.dtypes)

# print (results_df.groupby('scenario_id')['grid_elec_import_Wh'].sum())


# print (half_hourly_baseload_profile_df.groupby('scenario_id')['electricity_demand_baseload_Wh'].sum())

ttt = time.time()

merged_results_df = results_df_full

merged_results_df['grid_elec_import_cost'] = (merged_results_df['grid_elec_import_Wh']*
                                              merged_results_df['electricity_unit_rate_per_kWh']/1000.)

merged_results_df['electricity_baseload_import_cost'] = (merged_results_df['electricity_demand_baseload_Wh']*
                                                         merged_results_df['electricity_unit_rate_per_kWh']/1000.)

merged_results_df['electricity_ev_import_cost'] = (merged_results_df['ev_charging_demand_Wh']*
                                                   merged_results_df['electricity_unit_rate_per_kWh']/1000.)

merged_results_df['electricity_heatpump_import_cost'] = (merged_results_df['electricity_demand_heatpump_Wh']*
                                                         merged_results_df['electricity_unit_rate_per_kWh']/1000.)

merged_results_df['electricity_battery_storage_import_cost'] = (merged_results_df['battery_charging_demand_Wh']*
                                                                merged_results_df['electricity_unit_rate_per_kWh']/1000.)

merged_results_df['electricity_export_income'] = merged_results_df['grid_elec_export_Wh']*0.055/1000.
# merged_results_df['electricity_export_Wh']

merged_results_df['gas_import_cost'] = (merged_results_df['gas_demand_Wh']*
                                        merged_results_df['gas_unit_rate_per_kWh']/1000.)

print ('merged_results_df Mem Usage:',merged_results_df.memory_usage().sum()/1e9)
print ('Merging results with half-hourly baseload profile:',time.time()-ttt,'seconds')

print ('HH Baseload Profile Mem Usage:',half_hourly_baseload_profile_df.memory_usage().sum()/1e9)
print ('Pre-merge Results Mem Usage:',results_df.memory_usage().sum()/1e9)
print ('Merged Results (HH) Mem Usage:',merged_results_df.memory_usage().sum()/1e9)
print ('Scenario_df Results Mem Usage:',scenario_df.memory_usage().sum()/1e9)


# Create a summary results dataframe, that can be easily read in by Streamlit and plotted, needs to include:

# Costs, kWh usage for each component (Baseload, EV, Solar PV, Heat Pump, Battery Storage)
# Calculate cost, miles travelled, for ICE vehicle
# Calculate standing charge for each fuel type
# Calculate a cost for each fuel type
# Calculate a total cost

cols = ['grid_elec_import_Wh','grid_elec_export_Wh','electricity_demand_baseload_Wh',
        'ev_charging_demand_Wh','electricity_demand_heatpump_Wh','solar_pv_generation_Wh',
        'battery_charging_demand_Wh','battery_generation_Wh','gas_demand_Wh',
        'grid_elec_import_cost',
        'electricity_export_income','gas_import_cost'
        ]

summary_results_df = merged_results_df.groupby('scenario_id')[cols].sum().reset_index()

summary_results_df = pd.merge(summary_results_df, scenario_df, on='scenario_id')
summary_results_df['vehicle_litres_fuel_annual'] = summary_results_df['daily_miles_driven']*365*4.54609/summary_results_df['vehicle_miles_per_gallon']
summary_results_df.replace([np.inf, -np.inf], 0., inplace=True)
summary_results_df.to_csv('summary_results.csv', index=False)

summary_results_df

Pre-merge Results Mem Usage: 6135.64416
merged_results_df Mem Usage: 8.39614464
Merging results with half-hourly baseload profile: 10.29174280166626 seconds
HH Baseload Profile Mem Usage: 0.7926048
Pre-merge Results Mem Usage: 0.262379648
Merged Results (HH) Mem Usage: 8.39614464
Scenario_df Results Mem Usage: 0.000960768


,scenario_id,grid_elec_import_Wh,grid_elec_export_Wh,electricity_demand_baseload_Wh,ev_charging_demand_Wh,electricity_demand_heatpump_Wh,solar_pv_generation_Wh,battery_charging_demand_Wh,battery_generation_Wh,gas_demand_Wh,...,gas_standing_charge_daily,location_id,location_name,annual_electricity_consumption_kWh,annual_gas_consumption_kWh,daily_miles_driven,profile_id,electricity_profile_multiplier,electricity_demand_ev_daily_Wh,vehicle_litres_fuel_annual
0,0,2.200000e+06,0.000000e+00,2200000.0,0.0,0.000000e+00,0.00,0.000000e+00,0.000000e+00,10000000.0,...,0.2722,0,Thames,2200,10000,0,0,6.027397,0.0,0.000000
1,1,2.200000e+06,0.000000e+00,2200000.0,0.0,0.000000e+00,0.00,0.000000e+00,0.000000e+00,10000000.0,...,0.2722,0,Thames,2200,10000,20,0,6.027397,0.0,829.661425
2,2,2.200000e+06,0.000000e+00,2200000.0,0.0,0.000000e+00,0.00,0.000000e+00,0.000000e+00,10000000.0,...,0.2722,0,Thames,2200,10000,50,0,6.027397,0.0,2074.153563
3,3,2.200000e+06,0.000000e+00,2200000.0,0.0,0.000000e+00,0.00,0.000000e+00,0.000000e+00,10000000.0,...,0.2722,0,Thames,2200,10000,100,0,6.027397,0.0,4148.307125
4,4,2.200000e+06,0.000000e+00,2200000.0,0.0,0.000000e+00,0.00,0.000000e+00,0.000000e+00,12000000.0,...,0.2722,0,Thames,2200,12000,0,0,6.027397,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,2299,1.461155e+07,1.269994e+06,3600000.0,9125000.0,4.597701e+06,3981146.44,3.841098e+06,3.841098e+06,0.0,...,0.1437,0,Thames,3600,12000,100,0,9.863014,25000.0,0.000000
2300,2300,6.162978e+06,1.180140e+06,3600000.0,0.0,5.363985e+06,3981146.44,3.951726e+06,3.951726e+06,0.0,...,0.1437,0,Thames,3600,14000,0,0,9.863014,0.0,NaN
2301,2301,7.987978e+06,1.180140e+06,3600000.0,1825000.0,5.363985e+06,3981146.44,3.951726e+06,3.951726e+06,0.0,...,0.1437,0,Thames,3600,14000,20,0,9.863014,5000.0,0.000000
2302,2302,1.072548e+07,1.180140e+06,3600000.0,4562500.0,5.363985e+06,3981146.44,3.951726e+06,3.951726e+06,0.0,...,0.1437,0,Thames,3600,14000,50,0,9.863014,12500.0,0.000000
